In [1]:
#pip install requests
#pip install python-dotenv
#pip install beautifulsoup4
#pip install openai

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import openai

# Cargar variables del archivo .env
load_dotenv()

# Configurar tu clave de API de OpenAI
openai.api_key = os.getenv("API_KEY")

# Descargar sitio url con parametro de etiqueta teatro
url = "https://www.rosario.gob.ar/inicio/agenda/buscar?etiquetas[333]=333"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

# Extraer texto relevante
contenedor = soup.select_one(".vista-agenda-filtros-avanzados > .govuk-grid-row")

if contenedor:
    agenda = contenedor.get_text(strip=True)
else:
    print("❌ No se encontró el contenedor. Verifica el selector CSS.")
    agenda = ""

In [2]:
conversation = [
  {
    "role": "system",
    "content": (
        "Actuás como un asistente experto en eventos culturales y recreativos de la ciudad de Rosario. "
    "Tu tarea es identificar los títulos de obras de teatro presentes en el texto fuente. "
    "Debés mostrar los títulos **exactamente como aparecen en el texto original**, sin omitir ninguna palabra, aunque incluyan expresiones como 'Funciones de', 'Obra:', 'Presentación de', etc. "
    "No debés resumir, reformular, corregir, traducir ni seleccionar partes del título: **copialo tal cual aparece**. "
    "El título debe ser tomado del bloque de texto que da a entender que se trata de una obra de teatro. "
    "Ignorá horarios, precios, lugares o descripciones, y no extraigas frases sueltas que no parezcan títulos. "
    "Si hay dudas, incluí toda la frase, sin modificarla."
    )
  },
  {
    "role": "user",
    "content": f"""Texto fuente con posibles obras de teatro:

{agenda}

IMPORTANTE:
- Si el texto está vacío o no contiene ninguna obra de teatro, respondé exclusivamente con: 'Por ahora no hay obras de teatro en la Agenda'.
- Si hay una o más obras, mostrá sus títulos exactamente como aparecen.
- Mostrá como máximo los primeros 5 títulos, sin completar ni inventar si hay menos.
- Comenzá la respuesta con: 'Estas son las obras de teatro que te sugiero:'."""
  },
  {
    "role": "user",
    "content": "¿Qué obras de teatro hay en Rosario?"
  }
]

In [3]:
response = openai.ChatCompletion.create(
    model="gpt-4o",  # este es el nombre correcto del modelo "04-mini"
    messages=conversation,
    temperature=0,
    max_tokens=150
)

print(response["choices"][0]["message"]["content"])

Estas son las obras de teatro que te sugiero:
1. Babilonia
2. El Susto en La Orilla Infinita
3. Conexión Modo Avión
4. La Hora del Lobo
5. Pibes Bien
